In [40]:
import pandas as pd
train_df = pd.read_csv('data/train.csv', header=0)
test_df = pd.read_csv('data/test.csv', header=0)
y = train_df['Survived']
train_df = train_df.drop('Survived', axis=1)

train_df.head()
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [41]:
from sklearn.base import BaseEstimator, TransformerMixin
class  DesiredAttributes(BaseEstimator, TransformerMixin):
    def fit (self, X, y=None):
        self.undesirable_columns = ['PassengerId','Name', 'Ticket', 'Cabin']
        return self

    def transform(self, X, y=None):
        X.head()
        return X.drop(self.undesirable_columns, axis=1)

desiredAttributes = DesiredAttributes()
Xdrop = desiredAttributes.fit_transform(train_df)


In [42]:
class NumericAttributes(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.numericColumns = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.numericColumns]

numericAttributes = NumericAttributes()
Xnum = numericAttributes.fit_transform(Xdrop)



In [43]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipenum = Pipeline([
    ('numericAttributes', NumericAttributes()),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

X_numeric_clean = pipenum.fit_transform(Xnum)



In [44]:
class CategoricalAttributes(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.numericColumns = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.numericColumns]

categoricalAttributes = CategoricalAttributes()
Xcat = categoricalAttributes.fit_transform(Xdrop)

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

pipecat = Pipeline([
    ('CategoricalAttributes', CategoricalAttributes()),
    ('imputer', SimpleImputer(strategy='most_frequent' )),
    ('enconder', OneHotEncoder())
])

X_cat_clean = pipecat.fit_transform(Xdrop)
X_cat_clean

<891x5 sparse matrix of type '<class 'numpy.float64'>'
	with 1782 stored elements in Compressed Sparse Row format>

In [46]:
from sklearn.pipeline import FeatureUnion, Pipeline

pipeunion = FeatureUnion([
    ('pipenum', pipenum),
    ('pipecat', pipecat)
 ])

X = pipeunion.fit_transform(Xdrop)
X

<891x10 sparse matrix of type '<class 'numpy.float64'>'
	with 6237 stored elements in Compressed Sparse Row format>

In [47]:
from sklearn.pipeline import Pipeline

preprocessing = Pipeline([
    ('desiredAttributes', DesiredAttributes()),
    ('union', pipeunion)
])

XFull = preprocessing.fit_transform(train_df)
 

In [48]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
import numpy as np

X = train_df

pipetotal = Pipeline([
    ('preprocessing', preprocessing),
    ('arvore', DecisionTreeClassifier())
])
pipetotal.fit(X, y)
ypred = pipetotal.predict(X)

accuracy_score(y, ypred)
scores = cross_validate(pipetotal, X, y)

Xtest = preprocessing.fit_transform(test_df)
ypredtest = pipetotal.predict(Xtest)

print(np.mean(scores['test_score']))



AttributeError: head not found